In [1]:
import os
import sys
import warnings

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction
# from classification_models import SkLearnPerceptronModel, SkLearnSGDClassifier, EvaluationMetric
from classification_models import SkLearnModelFactory, EvaluationMetric

In [25]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
base_path = os.path.join(notebook_dir, '../data/dataset/')
file = os.path.join(base_path, 'train.json')
data = pd.read_json(file, lines=True)
data

,title,event_list,event_list_extraction,category,wiki_sections,source,time,type,atomic_time,LTF,STF,MCNC,MCNC_extract,MCAC,news_list
0,Americans with Disabilities Act of 1990,[1986: The National Council on Disability reco...,"[[{'Event': 'recommendation', 'Time': '1986', ...","[Americans with Disabilities Act of 1990| , 10...","[{'title': 'Introduction', 'text': ' The '''...",wikipedia,1990-07-26,Law and crime,"[1986, 1988, 1990-07-26, 1991, 1992-07, None, ...",[{'known_timeline': ['1986: The National Counc...,[{'known_timeline': ['1986: The National Counc...,[{'known_timeline': ['1986: The National Counc...,[{'known_timeline': [[{'Event': 'recommendatio...,"[{'question_type': 'Subject', 'known_timeline'...",NaN
1,American Airlines Flight 77,"[July 20, 2001: Hani Hanjour, a lead hijacker,...","[[{'Event': 'practice flight', 'Time': 'July 2...","[American Airlines Flight 77| , 2001 fires in ...","[{'title': 'Introduction', 'text': ' '''Am...",wikipedia,2002,Armed conflicts and attacks,"[2001-07-20, 2001-09-11, 2001-09-11, 2001-09-1...","[{'known_timeline': ['July 20, 2001: Hani Hanj...","[{'known_timeline': ['July 20, 2001: Hani Hanj...","[{'known_timeline': ['July 20, 2001: Hani Hanj...",[{'known_timeline': [[{'Event': 'practice flig...,"[{'question_type': 'Subject', 'known_timeline'...",NaN
2,Anti-Ballistic Missile Treaty,[1959-1960s: The United States and the Soviet ...,"[[{'Event': 'development', 'Time': '1959-1960s...","[1972 in the Soviet Union, 1972 in the United ...","[{'title': 'Introduction', 'text': ' The ''...",wikipedia,2002-06,International relations,"[1959, 1967, 1969, 1972-05-26, 1974, 1987, 199...",[{'known_timeline': ['1959-1960s: The United S...,[{'known_timeline': ['1959-1960s: The United S...,[{'known_timeline': ['1959-1960s: The United S...,"[{'known_timeline': [[{'Event': 'development',...","[{'question_type': 'Subject', 'known_timeline'...",NaN
3,Alaska Native Claims Settlement Act,"[1959: Alaska becomes a state, and section 4 o...","[[{'Event': 'become a state', 'Time': '1959', ...","[Alaska Native Claims Settlement Act| , United...","[{'title': 'Introduction', 'text': ' The '''Al...",wikipedia,1971-12,Law and crime,"[1959, 1964, 1966, 1966-10-18, 1967, 1967, 196...",[{'known_timeline': ['1959: Alaska becomes a s...,[{'known_timeline': ['1959: Alaska becomes a s...,[{'known_timeline': ['1959: Alaska becomes a s...,[{'known_timeline': [[{'Event': 'become a stat...,"[{'question_type': 'Time', 'known_timeline': [...",NaN
4,2001 United Kingdom general election,[February 2001: The foot-and-mouth outbreak st...,"[[{'Event': 'outbreak', 'Time': 'February 2001...","[2001 United Kingdom general election| , 2001 ...","[{'title': 'Introduction', 'text': ' Seat...",wikipedia,2001,Politics and elections,"[2001-02, 2001-04, 2001-05-22, 2001-05-16, 200...",[{'known_timeline': ['February 2001: The foot-...,[{'known_timeline': ['February 2001: The foot-...,[{'known_timeline': ['February 2001: The foot-...,"[{'known_timeline': [[{'Event': 'outbreak', 'T...","[{'question_type': 'Object', 'known_timeline':...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41419,Insurgency in Balochistan|||Terrorism in Pakis...,[November: Breakdown in peace talks between th...,"[[{'Event': 'breakdown', 'Time': 'November', '...",Armed conflicts and attacks,NaN,wiki_current_events,2023-03-06,Armed conflicts and attacks,"[None, None, None, None, None, None, None]",[{'known_timeline': ['November: Breakdown in p...,[],NaN,NaN,"[{'question_type': 'type/behavior', 'known_tim...","[{'date': '2023-03-06', 'summary': 'At least n..."
41420,Armenia-Russia relations|||2022-2023 blockade ...,[2016: Armed members of Sasna Tsrer stormed a ...,"[[{'Event': 'storm', 'Time': '2016', 'Location...",Law and crime,NaN,wiki_current_events,2023-01-09,Law and crime,"[2016, 2018, 2020, 2022, 2022-09, 2023-01-08, ...",[{'known_timeline': ['2016: Armed members of S...,[{'known_timeline': ['2016: Armed members of S...,NaN,NaN,"[{'question_type': 'O

In [3]:
data['event_list_extraction'].values[2]

[[{'Event': 'development',
   'Time': '1959-1960s',
   'Location': None,
   'Subject': 'United States',
   'Object': 'missile systems'}],
 [{'Event': 'announcement',
   'Time': '1967',
   'Location': None,
   'Subject': 'US',
   'Object': 'Sentinel system'}],
 [{'Event': 'begin',
   'Time': '1969',
   'Location': None,
   'Subject': 'Strategic Arms Limitation Talks (SALT I)',
   'Object': None}],
 [{'Event': 'signing',
   'Time': '1972-05-26',
   'Location': None,
   'Subject': 'United States and the Soviet Union',
   'Object': 'Anti-Ballistic Missile Treaty (ABM Treaty)'}],
 [{'Event': 'reduction',
   'Time': '1974',
   'Location': None,
   'Subject': '1974 Protocol',
   'Object': 'ABM sites',
   'Parameter': 'number of sites to one per party'}],
 [{'Event': 'withdraw opposition',
   'Time': '1987',
   'Location': None,
   'Subject': 'USSR',
   'Object': 'SDI'},
  {'Event': 'conclude',
   'Time': '1987',
   'Subject': 'USSR',
   'Object': 'SDI',
   'statement content': 'it poses no th

In [12]:
import os
import pandas as pd
import json

# Load the dataset
base_path = os.path.join(notebook_dir, '../data/dataset/')
file = os.path.join(base_path, 'train.json')
data = pd.read_json(file, lines=True)

# Helper function to check for 'prediction' in any string field
def contains_prediction(value):
    return isinstance(value, str) and 'prediction' in value.lower()

# Function to extract predictions from nested fields
def extract_predictions(row):
    predictions = []
    title = row.get('title', '')

    # Check event_list_extraction
    event_list = row.get('event_list_extraction', [])
    if isinstance(event_list, list):
        for event_group in event_list:
            if isinstance(event_group, list):
                for event in event_group:
                    if isinstance(event, dict) and any(contains_prediction(str(v)) for v in event.values()):
                        predictions.append({'title': title, 'event': event})

    # Check timeline fields
    for field in ['LTF', 'STF', 'MCNC', 'MCNC_extract']:
        field_data = row.get(field, [])
        if isinstance(field_data, list):
            for entry in field_data:
                if isinstance(entry, dict):
                    for subfield in ['known_timeline', 'label', 'true_event', 'choices']:
                        subdata = entry.get(subfield, [])
                        if isinstance(subdata, list):
                            for item in subdata:
                                if isinstance(item, dict) and any(contains_prediction(str(v)) for v in item.values()):
                                    predictions.append({'title': title, 'event': item})
                                elif contains_prediction(str(item)):
                                    predictions.append({'title': title, 'event': item})
                elif isinstance(entry, list):
                    for item in entry:
                        if isinstance(item, dict) and any(contains_prediction(str(v)) for v in item.values()):
                            predictions.append({'title': title, 'event': item})
                        elif contains_prediction(str(item)):
                            predictions.append({'title': title, 'event': item})
                elif contains_prediction(str(entry)):
                    predictions.append({'title': title, 'event': entry})

    return predictions

# Apply the extraction function
extracted_data = data.apply(lambda row: extract_predictions(row), axis=1)

In [19]:
all_predictions = []
for sublist in extracted_data:
    for pred in sublist:
        # print(pred)
        all_predictions.append(pred)
all_predictions

[{'title': '1992 United Kingdom general election',
  'event': {'Event': 'prediction',
   'Time': 'April 9, 1992',
   'Location': None,
   'Subject': 'BBC and ITV exit polls',
   'Object': 'hung parliament and seat distribution between Conservatives and Labour'}},
 {'title': '1992 United Kingdom general election',
  'event': [{'Event': 'prediction',
    'Time': 'April 9, 1992',
    'Location': None,
    'Subject': 'BBC and ITV exit polls',
    'Object': 'hung parliament and seat distribution between Conservatives and Labour'}]},
 {'title': '1992 United Kingdom general election',
  'event': [{'Event': 'prediction',
    'Time': 'April 9, 1992',
    'Location': None,
    'Subject': 'BBC and ITV exit polls',
    'Object': 'hung parliament and seat distribution between Conservatives and Labour'}]},
 {'title': '1992 United Kingdom general election',
  'event': [{'Event': 'prediction',
    'Time': 'April 9, 1992',
    'Location': None,
    'Subject': 'BBC and ITV exit polls',
    'Object': 'hu

In [21]:
def parse_event_data(data_list):
    """
    Parse event data from various formats into a flat structure.
    
    Args:
        data_list: List of dictionaries with 'title' and 'event' keys
        
    Returns:
        pandas DataFrame with columns: title, time, location, subject, object
    """
    rows = []
    
    for item in data_list:
        title = item.get('title')
        event = item.get('event')
        
        # Handle different event formats
        if isinstance(event, dict):
            # Single event as dictionary
            if event.get('Event') == 'prediction':
                rows.append({
                    'title': title,
                    'time': event.get('Time'),
                    'location': event.get('Location'),
                    'subject': event.get('Subject'),
                    'object': event.get('Object')
                })
        elif isinstance(event, list):
            # Multiple events in a list
            for e in event:
                if isinstance(e, dict) and e.get('Event') == 'prediction':
                    rows.append({
                        'title': title,
                        'time': e.get('Time'),
                        'location': e.get('Location'),
                        'subject': e.get('Subject'),
                        'object': e.get('Object')
                    })
        # Skip string events (they're not structured)
    
    return pd.DataFrame(rows)

In [26]:
parse_event_data(all_predictions)

,title,time,location,subject,object
0,1992 United Kingdom general election,"April 9, 1992",None,BBC and ITV exit polls,hung parliament and seat distribution between Conservatives and Labour
1,1992 United Kingdom general election,"April 9, 1992",None,BBC and ITV exit polls,hung parliament and seat distribution between Conservatives and Labour
2,1992 United Kingdom general election,"April 9, 1992",None,BBC and ITV exit polls,hung parliament and seat distribution between Conservatives and Labour
3,1992 United Kingdom general election,"April 9, 1992",None,BBC and ITV exit polls,hung parliament and seat distribution between Conservatives and Labour
4,Operation Castle,"April 22, 1954",None,None,yield of 1.0 Mt
5,Operation Castle,"April 22, 1954",None,None,yield of 1.0 Mt
6,Operation Castle,"April 22, 1954",None,None,yield of 1.0 Mt
7,Operation Castle,"April 22, 1954",None,None,yield of 1.0 Mt
8,Hurricane Fabian,November 2014,Bermuda,a catastrophe modeller,Hurricane Fabian
9,Hurricane Fabian,2014,None,a catastrophe modeller,Hurricane Fabian
